# Линейная регрессия. Работа с признаками

## Описание задачи и загрузка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

В этом задании мы рассмотрим различные аспекты построения линейной модели. Мы будем работать с одним из классических наборов данных в статистике, содержащим информацию о бриллиантах. Описание можно посмотреть [здесь](https://www.kaggle.com/shivam2503/diamonds).

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/diamonds.csv')
data.head(5)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Посмотрим на типы столбцов.

In [3]:
data.dtypes

,0
Unnamed: 0,int64
carat,float64
cut,object
color,object
clarity,object
depth,float64
table,float64
price,int64
x,float64
y,float64


Мы будем решать задачу предсказания цены бриллианта `price` в зависимости от его характеристик.

## Построение модели

### Задание 1

Есть ли в наборе данных пропущенные значения? Если да, удалите их.

Также выведите на экран число пропусков в каждом столбце.

In [7]:
print(data.isnull().sum())

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64


### Задача 2

Есть ли в наборе данных бессмысленные столбцы (признаки, не несущие дополнительной информации)?  
Если да, то удалите их.

In [8]:
data

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [9]:
data.drop(columns=["Unnamed: 0"], inplace=True)

### Задание 3

Линейная регрессия основана на предположении о линейной связи между признаками и целевой переменной, а потому перед выбором переменных для включения в модель имеет смысл проверить, насколько эта связь выполняется. Для следующих пунктов нам также потребуются корреляции между признаками.  
Выведите матрицу корреляций между всеми вещественными признаками и целевой переменной.

Какой вещественный признак коррелирует с целевой переменной больше всего?

In [12]:
df = data

In [15]:
import pandas as pd
import numpy as np

target_column = 'price'

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [col for col in numeric_cols if col != target_column]
numeric_df = df[numeric_features + [target_column]]


corr_with_target = numeric_df.corr()[target_column].drop(target_column)
corr_matrix = numeric_df.corr()
print("Матрица корреляций:")
print(corr_matrix)

max_corr_feature = corr_with_target.abs().idxmax()
max_corr_value = corr_with_target[max_corr_feature]

print(f"\nПризнак с наибольшей корреляцией с целевой переменной: {max_corr_feature}")
print(f"Корреляция: {max_corr_value}")

Матрица корреляций:
          carat     depth     table         x         y         z     price
carat  1.000000  0.028224  0.181618  0.975094  0.951722  0.953387  0.921591
depth  0.028224  1.000000 -0.295779 -0.025289 -0.029341  0.094924 -0.010647
table  0.181618 -0.295779  1.000000  0.195344  0.183760  0.150929  0.127134
x      0.975094 -0.025289  0.195344  1.000000  0.974701  0.970772  0.884435
y      0.951722 -0.029341  0.183760  0.974701  1.000000  0.952006  0.865421
z      0.953387  0.094924  0.150929  0.970772  0.952006  1.000000  0.861249
price  0.921591 -0.010647  0.127134  0.884435  0.865421  0.861249  1.000000

Признак с наибольшей корреляцией с целевой переменной: carat
Корреляция: 0.9215913011934687


### Задание 4

Так как линейная модель складывает значения признаков с некоторыми весами, нам нужно аккуратно обработать категориальные признаки. Закодируйте категориальные переменные при помощи OneHot-кодирования ([`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)). Не забудьте поставить значение параметра `drop_first` равным `True`.

Сколько получилось столбцов в таблице `data`?

*P.S. Числовые столбцы оставляем в таблице без изменений.*

In [16]:
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Исходное число столбцов: {data.shape[1]}")
print(f"Числовые столбцы: {numeric_cols}")
print(f"Категориальные столбцы: {categorical_cols}")

for col in categorical_cols:
    print(f"{col}: {data[col].nunique()} уникальных значений")

data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

print(f"\nЧисло столбцов после OneHot-кодирования: {data_encoded.shape[1]}")

Исходное число столбцов: 10
Числовые столбцы: ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
Категориальные столбцы: ['cut', 'color', 'clarity']
cut: 5 уникальных значений
color: 7 уникальных значений
clarity: 8 уникальных значений

Число столбцов после OneHot-кодирования: 24


### Задание 5

Создайте матрицу `X`, содержащую все признаки, и не содержащую целевую переменную `price`. Также создайте вектор `y`, содержащий целевую переменную `price`.

In [17]:
X, y = data_encoded.drop(columns=['price']), data_encoded['price']

Разделите выборку на тренировочную и тестовую. Долю тестовой выборки укажите равной `0.3`.

При разбиении укажите `random_state = 42`.

In [18]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

### Задание 6

Зачастую при использовании линейных моделей вещественные признаки масштабируются.  В этой задаче масштабируйте вещественные признаки тренировочной и тестовой выборок при помощи модуля `StandardScaler`.

*  Обучите (`fit`) scaler на тренировочных данных
*  Преобразуйте (`transform`) и трейн, и тест

После применения масштабирования матрица перестает быть объектом `pandas.DataFrame` - решите эту проблему.

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [21]:
scaler.fit(Xtrain)

StandardScaler()

In [22]:
Xtrain_norm = scaler.transform(Xtrain)
Xtest_norm = scaler.transform(Xtest)

### Задание 7

Обучите линейную регрессию на тренировочной выборке. Выведите *r2-score* на тренировочной и тестовой выборках.

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(Xtrain_norm, ytrain)

LinearRegression()

In [24]:
y_train_pred = model.predict(Xtrain_norm)
y_test_pred = model.predict(Xtest_norm)

# 3. R2-score
train_r2 = r2_score(ytrain, y_train_pred)
test_r2 = r2_score(ytest, y_test_pred)

print(f"R2-score на тренировочной выборке: {train_r2:.4f}")
print(f"R2-score на тестовой выборке: {test_r2:.4f}")

R2-score на тренировочной выборке: 0.9196
R2-score на тестовой выборке: 0.9202


### Задание 8

Выведите на экран веса, которые линейная регрессия присвоила признакам.

Какой признак имеет наибольший отрицательный вес? (наибольший по модулю среди всех отрицательных весов)

In [29]:
coefficients = model.coef_
intercept = model.intercept_

weights_df = pd.DataFrame({
    'Признак': data_encoded.drop('price', axis=1).columns,
    'Вес': coefficients
})

print("Веса модели:")
print(weights_df)
print(f"\nСвободный член (intercept): {intercept}")

Веса модели:
          Признак          Вес
0           carat  5338.615671
1           depth   -90.173817
2           table   -60.332280
3               x -1100.418850
4               y    -6.458917
5               z   -34.258945
6        cut_Good   170.602933
7       cut_Ideal   414.369515
8     cut_Premium   339.239204
9   cut_Very Good   310.654256
10        color_E   -83.640021
11        color_F  -104.560993
12        color_G  -202.863564
13        color_H  -362.002300
14        color_I  -442.783567
15        color_J  -529.449216
16     clarity_IF   975.933641
17    clarity_SI1  1611.571681
18    clarity_SI2  1042.921215
19    clarity_VS1  1670.318496
20    clarity_VS2  1817.664208
21   clarity_VVS1  1284.809475
22   clarity_VVS2  1461.034194

Свободный член (intercept): 3951.4953122517086


## Попытка улучшить качество модели

Данная часть является творческой, и задания в ней немного сложнее, чем предыдущие.

Следующие вопросы не проверяются тестами.

### Задание 9

Как можно заметить из анализа корреляционной матрицы в задании 3, между некоторыми признаками имеется сильная корреляция, что может быть индикатором проблемы *мультиколлинеарности*. Различия в порядке коэффициентов, выявленные в предыдущей задаче, также свидетельствуют об этом. Для решения этой проблемы можно либо исключить некоторые признаки из модели (например, если признак линейно зависим с какими-то другими, его можно исключить из модели, т.е. удалить из матрицы объект-признак и заново обучить модель).

Удалите из матриц `Xtrain` и `Xtest` признак, который наиболее сильно коррелирует с остальными. Заново обучите модель и оцените её качество. Улучшилось ли качество модели?

Попробуйте удалить какой-то другой признак (можете попробовать несколько вариантов). Помогло ли это улучшить качество модели?

In [ ]:
# your code here

### Задание 10

Иногда генерация новых признаков помогает модели лучше находить взаимосвязи между целевой переменной и признаками. Попробуйте придумать новые признаки и добавить их в модель.  

Помогло ли это улучшить качество модели?

In [ ]:
# your code here